In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
train = pd.read_csv('csv/train.csv')
valid = pd.read_csv("csv/test.csv")

In [3]:
train = train.dropna(axis='columns')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4651 entries, 0 to 4650
Data columns (total 10 columns):
title               4651 non-null object
description         4651 non-null object
plot                4651 non-null object
csm_review          4651 non-null object
need_to_know        4651 non-null object
csm_rating          4651 non-null int64
Genre               4651 non-null object
Book type           4651 non-null object
Publication date    4651 non-null object
Last updated        4651 non-null object
dtypes: int64(1), object(9)
memory usage: 363.5+ KB


In [4]:
labelencoder = LabelEncoder()
train.loc[:, "genre_cat"] = labelencoder.fit_transform(train.loc[:, "Genre"])
train.loc[:, "book_type_cat"] = labelencoder.fit_transform(train.loc[:, "Book type"])

In [7]:
#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(train['title']))


#converting text into integer sequences
train["title_seq"]  = tokenizer.texts_to_sequences(train['title']) 

print(len(max(train["title_seq"] , key=len)))
print(len(min(train["title_seq"] , key=len)))


27
1


In [33]:
train[["title", "title_seq"]].head()

,title,title_seq
0,"Bloodhound: Beka Cooper, Book 2","[2176, 947, 699, 3, 7]"
1,"The School for Good and Evil, Book 1","[1, 22, 12, 65, 5, 323, 3, 6]"
2,"Twilight: The Twilight Saga, Book 1","[458, 1, 458, 216, 3, 6]"
3,The Lost Girl,"[1, 39, 17]"
4,"Warcross, Book 1","[2177, 3, 6]"


In [24]:
#preparing vocabulary
tokenizer.fit_on_texts(list(train['description']))


#converting text into integer sequences
train["description_seq"]  = tokenizer.texts_to_sequences(train['description']) 

print(len(max(train["description_seq"] , key=len)))
print(len(min(train["description_seq"] , key=len)))

14
3


In [34]:
train[["description","description_seq"]].head()

,description,description_seq
0,Second book in fantasy series is more mature t...,"[312, 18, 6, 108, 56, 7, 44, 360, 85, 59]"
1,Fractured fairy tale has plenty of twists for ...,"[4506, 397, 69, 36, 330, 5, 862, 11, 108, 241]"
2,Overlong but engrossing popular vampire romance.,"[4710, 13, 1928, 345, 943, 163]"
3,"Engrossing story of an ""echo"" designed to repl...","[1928, 23, 5, 25, 7361, 2324, 4, 7090, 424, 72]"
4,Winning teen girl drives cyberpunk virtual rea...,"[496, 96, 72, 2436, 10966, 3597, 816, 616]"


In [9]:
embedding_columns = ['title', 'description', 'plot', 'csm_review', 'need_to_know']

x_tr, y_tr = train[embedding_columns].values, train['csm_rating'].values
x_val, y_val = valid[embedding_columns].values, valid["csm_rating"].values

In [22]:
x_tr

array([['Bloodhound: Beka Cooper, Book 2',
        'Second book in fantasy series is more mature than first.',
        'BLOODHOUND finds young policewoman -- aka "Dog" -- Beka Cooper taking on her first big case as an official Guardswoman. With an increasing number of forged coins (called "coles") appearing on the streets of Corus and rumors of a bad grain harvest starting to circulate, the country\'s entire economy could be in trouble. Accompanied by a grumpy pigeon and an enthusiastic scent hound named Achoo, Beka and her partner follow the coles\' trail to Port Caynn, where they meet a whole cast of new characters -- and encounter plenty of new trouble.\n',
        'Like its predecessor, Terrier, Bloodhound is much more of a crime procedural than a traditional fantasy novel. While magic certainly has a role in the story (Beka can talk to ghosts carried by pigeons, as well as pick up snippets of conversation from small whirlwinds), it\'s far less central than in Pierce\'s other books